# Spotify Ontario Data Collection

This notebook is responsible for collecting data from Spotify's API focusing on Ontario-based playlists and tracks. We'll set up the authentication, create data collection functions, and store the raw data for further processing.

## Setup Environment and Authentication

First, we'll set up our environment by importing required libraries and configuring the Spotify API authentication.

In [1]:
# Import required libraries
import os
import sys
import json
import time
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

print("Python path:", sys.executable)
print("Python version:", sys.version)

Python path: c:\Users\felip\Documents\DSMM_TERM_4\Project_Week_2\spotify-ontario-analysis\venv\Scripts\python.exe
Python version: 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]


In [2]:
# Load environment variables
load_dotenv()

def test_spotify_connection():
    """Test the Spotify API connection and credentials"""
    try:
        # Test with a simple search
        test_results = sp.search(q='test', type='track', limit=1)
        if test_results and 'tracks' in test_results:
            print("✅ Spotify API connection successful")
            return True
        else:
            print("❌ API response format unexpected")
            print("Response:", test_results)
            return False
    except Exception as e:
        print("❌ Spotify API connection failed")
        print("Error:", str(e))
        return False

# Configure Spotify client
client_credentials_manager = SpotifyClientCredentials(
    client_id=os.getenv('SPOTIFY_CLIENT_ID'),
    client_secret=os.getenv('SPOTIFY_CLIENT_SECRET')
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Test connection
if not test_spotify_connection():
    raise Exception("Failed to connect to Spotify API. Please check your credentials.")

✅ Spotify API connection successful


## Create Data Collection Functions

We'll define helper functions to collect playlist data, track information, and audio features from Spotify API.

In [3]:
def handle_rate_limit():
    """Handle rate limiting by adding delays"""
    time.sleep(2)  # Base delay between requests

def log_failed_ids(track_ids, reason):
    """Log failed track IDs to a file with timestamp and reason"""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_dir = os.path.join('..', 'logs')
    os.makedirs(log_dir, exist_ok=True)
    
    log_file = os.path.join(log_dir, 'failed_ids.log')
    with open(log_file, 'a') as f:
        f.write(f"\n=== Failed tracks at {timestamp} ===\n")
        f.write(f"Reason: {reason}\n")
        for track_id in track_ids:
            f.write(f"{track_id}\n")
        f.write("=" * 50 + "\n")

def retry_auth():
    """Retry authentication with new token"""
    try:
        print("🔄 Refreshing authentication token...")
        client_credentials_manager = SpotifyClientCredentials(
            client_id=os.getenv('SPOTIFY_CLIENT_ID'),
            client_secret=os.getenv('SPOTIFY_CLIENT_SECRET')
        )
        global sp
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
        return True
    except Exception as e:
        print(f"❌ Failed to refresh token: {str(e)}")
        return False

def get_single_track_features(track_id):
    """Get audio features for a single track with error handling"""
    try:
        handle_rate_limit()
        features = sp.audio_features([track_id])
        if features and features[0]:
            return features[0]
    except Exception as e:
        print(f"❌ Failed to get features for track {track_id}: {str(e)}")
    return None

def get_audio_features(track_ids):
    """
    Get audio features for a list of tracks with improved error handling and recovery
    """
    if not track_ids:
        return []
    
    # Convert set to list if needed
    track_ids = list(track_ids)
    chunks = [track_ids[i:i + 10] for i in range(0, len(track_ids), 10)]
    
    audio_features = []
    failed_ids = set()
    total_chunks = len(chunks)
    processed = 0
    
    print(f"\n=== Starting Audio Features Collection ===")
    print(f"Total tracks to process: {len(track_ids)} in {total_chunks} chunks")
    
    for i, chunk in enumerate(chunks, 1):
        retry_count = 0
        max_retries = 3
        
        while retry_count < max_retries:
            try:
                print(f"\nProcessing chunk {i}/{total_chunks} ({len(chunk)} tracks)")
                handle_rate_limit()
                features = sp.audio_features(chunk)
                
                if features and any(features):
                    valid_features = [f for f in features if f is not None]
                    audio_features.extend(valid_features)
                    processed += len(valid_features)
                    
                    # Track failed IDs in this chunk
                    if len(valid_features) < len(chunk):
                        failed_in_chunk = set(chunk) - {f['id'] for f in valid_features}
                        failed_ids.update(failed_in_chunk)
                    
                    print(f"✅ Got {len(valid_features)}/{len(chunk)} features (Total: {processed}/{len(track_ids)} - {processed/len(track_ids)*100:.1f}%)")
                    break  # Success, move to next chunk
                
                else:
                    print("⚠️ No valid features in chunk, will retry individual tracks")
                    failed_ids.update(chunk)
                    break
                    
            except Exception as e:
                error_msg = str(e).lower()
                
                if '403' in error_msg or 'forbidden' in error_msg or 'token' in error_msg:
                    print("🔑 Auth error detected, attempting to refresh token...")
                    if retry_auth():
                        retry_count += 1
                        continue
                    else:
                        print("❌ Token refresh failed, will try individual tracks")
                        failed_ids.update(chunk)
                        break
                else:
                    print(f"❌ Error processing chunk: {str(e)}")
                    retry_count += 1
                    if retry_count < max_retries:
                        print(f"Retrying in 5 seconds... (Attempt {retry_count + 1}/{max_retries})")
                        time.sleep(5)
                    else:
                        print("Max retries reached, will try individual tracks")
                        failed_ids.update(chunk)
                        break
    
    # Try to recover failed tracks individually
    if failed_ids:
        print(f"\n🔄 Attempting to recover {len(failed_ids)} failed tracks individually...")
        recovered = 0
        
        for track_id in failed_ids.copy():
            feature = get_single_track_features(track_id)
            if feature:
                audio_features.append(feature)
                failed_ids.remove(track_id)
                recovered += 1
                print(f"✅ Recovered track {track_id}")
            else:
                print(f"❌ Failed to recover track {track_id}")
        
        if recovered:
            print(f"\n✅ Successfully recovered {recovered} tracks")
    
    # Log any remaining failed IDs
    if failed_ids:
        print(f"\n⚠️ Could not process {len(failed_ids)} tracks")
        log_failed_ids(failed_ids, "Failed after individual recovery attempts")
    
    final_success_rate = (len(audio_features) / len(track_ids)) * 100
    print(f"\n=== Collection Summary ===")
    print(f"Total tracks processed: {len(audio_features)}/{len(track_ids)} ({final_success_rate:.1f}%)")
    print(f"Failed tracks: {len(failed_ids)}")
    
    return audio_features

def search_ontario_playlists(query="Ontario", limit=50):
    """Search for Ontario-related playlists on Spotify with error handling"""
    try:
        handle_rate_limit()
        results = sp.search(q=query, type='playlist', limit=limit)
        if not results or 'playlists' not in results:
            print("❌ No results found or invalid response format")
            return []
            
        items = results['playlists']['items']
        if not items:
            print("ℹ️ No playlists found for query:", query)
            return []
            
        print(f"✅ Found {len(items)} playlists")
        return items
    except Exception as e:
        print("❌ Error searching playlists:", str(e))
        return []

def get_playlist_tracks(playlist_id):
    """Get all tracks from a playlist with error handling"""
    try:
        handle_rate_limit()
        results = sp.playlist_tracks(playlist_id)
        if not results or 'items' not in results:
            print(f"❌ Invalid response for playlist {playlist_id}")
            return []
            
        tracks = results['items']
        
        while results.get('next'):
            try:
                handle_rate_limit()
                results = sp.next(results)
                if results and 'items' in results:
                    tracks.extend(results['items'])
            except Exception as e:
                print(f"❌ Error fetching next page: {str(e)}")
                break
        
        print(f"✅ Retrieved {len(tracks)} tracks from playlist {playlist_id}")
        return tracks
    except Exception as e:
        print(f"❌ Error getting tracks for playlist {playlist_id}:", str(e))
        return []

def extract_track_info(track_item):
    """Extract relevant track information from API response with error handling"""
    try:
        if not isinstance(track_item, dict) or 'track' not in track_item:
            return None
            
        track = track_item['track']
        if not track or not isinstance(track, dict) or 'id' not in track:
            return None
            
        artists = track.get('artists', [])
        artist_name = artists[0].get('name', 'Unknown') if artists else 'Unknown'
        
        album = track.get('album', {})
        album_name = album.get('name', 'Unknown')
        release_date = album.get('release_date')
            
        return {
            'track_id': track['id'],
            'name': track.get('name', 'Unknown'),
            'artist': artist_name,
            'album': album_name,
            'release_date': release_date,
            'popularity': track.get('popularity', 0),
            'duration_ms': track.get('duration_ms', 0),
            'explicit': track.get('explicit', False)
        }
    except Exception as e:
        print("❌ Error extracting track info:", str(e))
        return None

## Extract Playlist Data

Now we'll use our functions to collect playlist data and track information.

In [4]:
# Search for Ontario playlists
ontario_playlists = search_ontario_playlists()

if ontario_playlists:
    # Extract basic playlist information
    playlists_data = []
    for playlist in ontario_playlists:
        try:
            # Verify that we have a valid playlist dictionary
            if not isinstance(playlist, dict):
                continue
                
            # Safely get nested values with fallbacks
            owner_info = playlist.get('owner', {})
            tracks_info = playlist.get('tracks', {})
            followers_info = playlist.get('followers', {})
                
            playlist_info = {
                'playlist_id': playlist.get('id', ''),
                'name': playlist.get('name', 'Unknown'),
                'description': playlist.get('description', ''),
                'owner': owner_info.get('display_name', 'Unknown'),
                'tracks_total': tracks_info.get('total', 0),
                'followers': followers_info.get('total', 0)
            }
            
            # Only add if we have a valid playlist ID
            if playlist_info['playlist_id']:
                playlists_data.append(playlist_info)
        except Exception as e:
            playlist_name = playlist.get('name', 'Unknown') if isinstance(playlist, dict) else 'Unknown'
            print(f"❌ Error processing playlist {playlist_name}: {str(e)}")
            continue

    # Convert to DataFrame
    playlists_df = pd.DataFrame(playlists_data)
    print(f"✅ Successfully processed {len(playlists_df)} playlists")
else:
    print("❌ No playlists to process")
    playlists_df = pd.DataFrame()

✅ Found 49 playlists
✅ Successfully processed 38 playlists


In [5]:
# Collect tracks from each playlist
all_tracks = []
track_ids = set()

if not playlists_df.empty:
    for playlist_id in playlists_df['playlist_id']:
        tracks = get_playlist_tracks(playlist_id)
        
        # Extract track information
        for item in tracks:
            track_info = extract_track_info(item)
            if track_info:
                track_info['playlist_id'] = playlist_id
                all_tracks.append(track_info)
                track_ids.add(track_info['track_id'])

    # Convert to DataFrame
    tracks_df = pd.DataFrame(all_tracks)
    print(f"✅ Collected {len(tracks_df)} tracks ({len(track_ids)} unique)")
else:
    print("❌ No playlists available to collect tracks from")
    tracks_df = pd.DataFrame()

✅ Retrieved 280 tracks from playlist 3vGg6v4ZaYhhYZOCvXVUCx
✅ Retrieved 702 tracks from playlist 7sn0BpUrX5ptjFFzGhRy9Z
✅ Retrieved 544 tracks from playlist 5SUabhFE7utj8j7zoBOpVI
✅ Retrieved 861 tracks from playlist 32uH7sB8TAl03OtVOUuJzy
✅ Retrieved 578 tracks from playlist 5veRytqDxeCebNaaNT5b7S
✅ Retrieved 625 tracks from playlist 0myFM9AwR26t2zaX3tjrPW
✅ Retrieved 344 tracks from playlist 3n8N9K3Xyxx85nkrQj3HOt
✅ Retrieved 290 tracks from playlist 4jLRvvKZ5xKPxdxGrRAL63
✅ Retrieved 300 tracks from playlist 6RFzWKNpvGOFcEwZs69XmW
✅ Retrieved 151 tracks from playlist 5CciES5SR6E64QLNjcQZG0
✅ Retrieved 326 tracks from playlist 43AMl7KE9AdEBoxZSflhQB
✅ Retrieved 22 tracks from playlist 2Lne3xKmqPCrcuJqOfQpdr
✅ Retrieved 21 tracks from playlist 6hItlHywbIHpv4vLcO3GNl
✅ Retrieved 116 tracks from playlist 3UPfuFzaCB3HCSwAOOGVmY
✅ Retrieved 10 tracks from playlist 6wT6kcSOKPd88IgmG6Oj8F
✅ Retrieved 16 tracks from playlist 66XDl402XcPVosSaF8LONU
✅ Retrieved 1071 tracks from playlist 67mcnh

In [ ]:
# Get audio features for unique tracks
if track_ids:
    print(f"\n=== Starting Audio Features Collection ===")
    print(f"Total unique tracks to process: {len(track_ids)}")
    audio_features = get_audio_features(list(track_ids))

    # Convert to DataFrame with error handling
    if audio_features:
        audio_features_df = pd.DataFrame(audio_features)
        success_rate = (len(audio_features_df) / len(track_ids)) * 100
        print(f"\n✅ Successfully collected audio features for {len(audio_features_df)}/{len(track_ids)} tracks ({success_rate:.1f}%)")
    else:
        print("\n❌ No audio features collected")
        audio_features_df = pd.DataFrame()
else:
    print("\nℹ️ No track IDs to process")
    audio_features_df = pd.DataFrame()


=== Starting Audio Features Collection ===
Total unique tracks to process: 14910

=== Starting Audio Features Collection ===
Total tracks to process: 14910 in 1491 chunks

Processing chunk 1/1491 (10 tracks)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=15uLWl8au1kIyJaCtgKaYI,5SIuyYZIVW9WU90puvFG1x,0QRPnkRK35yM6wYCQ1seo9,02Ya2Q08bBKFqUJvjmyRXK,15JINEqzVMv3SvJTAXAKED,1xEcsGNqeAeeB5Yx2gmFRJ,6EKrBVHDk20hwjYWGPQQlF,7vrwS8Z8JOET8eVpSbAKHM,0N0h5lXc4HOiLgGb9EZi8m,5nHASDdR9qnAzUInjUlzUK with Params: {} returned 403 due to None


🔑 Auth error detected, attempting to refresh token...
🔄 Refreshing authentication token...

Processing chunk 1/1491 (10 tracks)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=15uLWl8au1kIyJaCtgKaYI,5SIuyYZIVW9WU90puvFG1x,0QRPnkRK35yM6wYCQ1seo9,02Ya2Q08bBKFqUJvjmyRXK,15JINEqzVMv3SvJTAXAKED,1xEcsGNqeAeeB5Yx2gmFRJ,6EKrBVHDk20hwjYWGPQQlF,7vrwS8Z8JOET8eVpSbAKHM,0N0h5lXc4HOiLgGb9EZi8m,5nHASDdR9qnAzUInjUlzUK with Params: {} returned 403 due to None


🔑 Auth error detected, attempting to refresh token...
🔄 Refreshing authentication token...

Processing chunk 1/1491 (10 tracks)


...

🔑 Auth error detected, attempting to refresh token...
🔄 Refreshing authentication token...

Processing chunk 98/1491 (10 tracks)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2M8IvrzqvNMAJsAx7UYwXK,7GuoIxCccDF44lzRdhsycf,65l3mI7YNuatZ9dYbJi89H,4jacsL77ZYnpInmTtUBaJW,067ogyD1BIeZDh4UD0yV3Y,5m1YsTxa9oSdmNVmgX1J7Y,6kls8cSlUyHW2BUOkDJIZE,0OLNfqtgqTDv4KeCzVrSH5,3glxhrtQiAeNtla0vtxEo2,0OQZiRDO3ymmhUpzrFY0dr with Params: {} returned 403 due to None


🔑 Auth error detected, attempting to refresh token...
🔄 Refreshing authentication token...

Processing chunk 98/1491 (10 tracks)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2M8IvrzqvNMAJsAx7UYwXK,7GuoIxCccDF44lzRdhsycf,65l3mI7YNuatZ9dYbJi89H,4jacsL77ZYnpInmTtUBaJW,067ogyD1BIeZDh4UD0yV3Y,5m1YsTxa9oSdmNVmgX1J7Y,6kls8cSlUyHW2BUOkDJIZE,0OLNfqtgqTDv4KeCzVrSH5,3glxhrtQiAeNtla0vtxEo2,0OQZiRDO3ymmhUpzrFY0dr with Params: {} returned 403 due to None


🔑 Auth error detected, attempting to refresh token...
🔄 Refreshing authentication token...

Processing chunk 99/1491 (10 tracks)


## Save Raw Data

Save the collected data to files in the raw data directory.

In [ ]:
# Create data directory if it doesn't exist
data_dir = os.path.join('..', 'data', 'raw')
os.makedirs(data_dir, exist_ok=True)

# Create timestamp for file names
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

try:
    # Save playlists data
    if not playlists_df.empty:
        playlists_file = os.path.join(data_dir, f'playlists_{timestamp}.csv')
        playlists_df.to_csv(playlists_file, index=False)
        print(f"✅ Saved {len(playlists_df)} playlists to {playlists_file}")

    # Save tracks data
    if not tracks_df.empty:
        tracks_file = os.path.join(data_dir, f'tracks_{timestamp}.csv')
        tracks_df.to_csv(tracks_file, index=False)
        print(f"✅ Saved {len(tracks_df)} tracks to {tracks_file}")

    # Save audio features data
    if not audio_features_df.empty:
        features_file = os.path.join(data_dir, f'audio_features_{timestamp}.csv')
        audio_features_df.to_csv(features_file, index=False)
        print(f"✅ Saved {len(audio_features_df)} audio features to {features_file}")
    
    print(f"\nData collection completed successfully at {timestamp}")
except Exception as e:
    print(f"❌ Error saving data: {str(e)}")